In [1]:
# Pré-requisitos:
# pip install pandas scikit-learn pyarrow mlflow dvc

import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import pickle
import os
import mlflow
import mlflow.sklearn

### A.1 Carregamento e Análise Preliminar dos Dados

In [2]:
import pandas as pd
import os

# Caminho base
caminho_base = "E:/Mestrado UFCG/Semestre 2024.2/Dados/Tabelas_0/"
arquivo_alunos = os.path.join(caminho_base, "alunos.csv")
arquivo_matriculas = os.path.join(caminho_base, "matriculas.csv")

# Verificar se os arquivos existem
print("Arquivo alunos existe?", os.path.exists(arquivo_alunos))
print("Arquivo matriculas existe?", os.path.exists(arquivo_matriculas))

# Tentativa de leitura com parâmetros mais seguros
try:
    df_alunos = pd.read_csv(arquivo_alunos, encoding="utf-8", sep=",")
    print("Tabela 'alunos' carregada com sucesso!")
except Exception as e:
    print("Erro ao carregar 'alunos':", e)

try:
    df_matriculas = pd.read_csv(arquivo_matriculas, encoding="utf-8", sep=",")
    print("Tabela 'matriculas' carregada com sucesso!")
except Exception as e:
    print("Erro ao carregar 'matriculas':", e)


Arquivo alunos existe? True
Arquivo matriculas existe? True
Tabela 'alunos' carregada com sucesso!
Erro ao carregar 'matriculas': Error tokenizing data. C error: Expected 1 fields in line 83624, saw 2



### A.2 Construção do Pipeline de Pré-processamento

Utiliza-se scikit-learn para construir um pipeline que normaliza as variáveis numéricas. O pipeline é salvo utilizando a biblioteca pickle.

In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import pickle

numericas = df_alunos.select_dtypes(include=['int64', 'float64']).columns
X = df_alunos[numericas].dropna()

pipeline = Pipeline([
    ('scaler', StandardScaler())
])

X_transformado = pipeline.fit_transform(X)

# Salvando pipeline
with open('preprocessador.pkl', 'wb') as f:
    pickle.dump(pipeline, f)

ValueError: at least one array or dtype is required

### A.3 Armazenamento da Feature Store

In [ ]:
pd.DataFrame(X_transformado, columns=numericas).to_csv("feature_store.csv", index=False)
pd.DataFrame(X_transformado, columns=numericas).to_parquet("feature_store.parquet")

### A.4 Registro de Experimentos com MLflow

O MLflow é utilizado para logar os artefatos, parâmetros e pipeline utilizado.

In [ ]:
import mlflow
import mlflow.sklearn

mlflow.set_experiment("Evasao_Aluno_UFCG")
with mlflow.start_run(run_name="Preprocessamento e Feature Store"):
    mlflow.log_artifact("feature_store.csv")
    mlflow.log_artifact("feature_store.parquet")
    mlflow.log_artifact("preprocessador.pkl")
    mlflow.log_param("scaler", "StandardScaler")
    mlflow.sklearn.log_model(pipeline, artifact_path="modelo_pipeline")

### A.5 Versionamento com DVC

Com os arquivos salvos, o controle de versão é feito utilizando o DVC. Abaixo, comandos a serem executados no terminal:

In [ ]:
### Para verificar o status:

In [ ]:
dvc status
dvc dag